In [1]:
import ghidra_bridge

# Activate Ghidra bridge, and use global namespace
bridge = ghidra_bridge.GhidraBridge(namespace=globals())

In [2]:
from itertools import *
        
BasicBlockModel = ghidra.program.model.block.BasicBlockModel
TaskMonitor = ghidra.util.task.TaskMonitor

bbm = BasicBlockModel(currentProgram)
blocks = bbm.getCodeBlocks(TaskMonitor.DUMMY)

for i in islice(blocks, 10):
    print(i)

LAB_00003026  src:[]  dst:[]
00004232  src:[]  dst:[0000426e, 0000424e]
0000424e  src:[00004232]  dst:[0000426e]
LAB_0000426e  src:[00004232, 0000424e]  dst:[]
00004e00  src:[]  dst:[00004e04, 00004e02]
00004e02  src:[00004e00]  dst:[00004e04]
LAB_00004e04  src:[00004e00, 00004e02]  dst:[00004e08, 00004e06]
00004e06  src:[00004e04]  dst:[00004e08]
LAB_00004e08  src:[00004e04, 00004e06]  dst:[00004e0c, 00004e0a]
00004e0a  src:[00004e08]  dst:[00004e0c]


In [15]:
Address = ghidra.program.model.address.Address
GenericAddress = ghidra.program.model.address.GenericAddress

FlatProgramAPI = ghidra.program.flatapi.FlatProgramAPI

load_inst_addr = toAddr(0x0019e778)  # a load instruction used before ptr deref, which we will base off of.
ptr_imm_addr = toAddr(0x0019e7e8)  # a 32-bit immediate, stored in ROM, holding the ptr location

papi = FlatProgramAPI(currentProgram)

inst = papi.getInstructionAt(load_inst_addr)
print(str(inst))
print('mnemonic:', inst.getMnemonicString())

inputs = inst.getInputObjects()  # the address
print('input objects:', list(inputs))
print('refers to the ptr we expect?', args[0] == ptr_imm_addr)
print()

results = inst.getResultObjects()[0]
print('result objects:', results)  # a register

ldr r3,[0x0019e7e8]
mnemonic: ldr
input objects: [<_bridged_ghidra.program.model.address.GenericAddress('0019e7e8', type=ghidra.program.model.address.GenericAddress, handle=9dc6044a-fe2b-4f28-8310-1d4bbf03af2f)>]
refers to the ptr we expect? True

result objects: r3


In [17]:
# Find the next use of the address (preferably a load)
block = bbm.getFirstCodeBlockContaining(inst.getAddress(), TaskMonitor.DUMMY)
block.

<_bridged_ghidra.program.model.block.CodeBlockImpl('0019e776  src:[0019e764]  dst:[001978b8, 0019e7d8, 0019e798]', type=ghidra.program.model.block.CodeBlockImpl, handle=6021dff9-7c61-4ea8-96cf-ce83b65dd952)>

In [4]:
data = papi.getDataAt(ptr_imm_addr)
print(repr(data.getValue()))
refFrom = data.getReferencesFrom()[0]
refFrom

<_bridged_ghidra.program.model.address.GenericAddress('00038415', type=ghidra.program.model.address.GenericAddress, handle=dae9a168-5c8c-40d3-a9a7-fe4390a63b88)>


<_bridged_ghidra.program.database.references.MemReferenceDB('From: 0019e7e8 To: 00038415 Type: DATA Op: 0 DEFAULT', type=ghidra.program.database.references.MemReferenceDB, handle=568fa815-3a6e-4043-a5bd-e734b280f61c)>

In [5]:
refTo = list(data.getReferenceIteratorTo())[0]

In [6]:
inst.

<_bridged_ghidra.program.database.code.InstructionDB('ldr r3,[0x0019e7e8]', type=ghidra.program.database.code.InstructionDB, handle=777c2bfa-e260-435e-b59d-7b51419e9fff)>

In [15]:
all_word_loads = bridge.remote_eval('''
[
    i
    for i in currentProgram.getListing().getInstructions(True)
    if i.getMnemonicString() == 'ldr'
]
''', papi=papi)

KeyboardInterrupt: 

In [ ]:
all_word_loads